In [138]:
import pandas as pd
from pprint import pprint
import json
import csv
import sys

In [3]:
#mounting the drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [139]:
#load the json file
f = open('/content/drive/MyDrive/Data_Science/Adnene task/phil.coutinho-1.json')
data_json = json.load(f)
f.close()

In [140]:
#extra step to give a summary
df = pd.io.json.json_normalize(data_json, record_path="GraphImages")[["edge_media_preview_like.count", "edge_media_to_comment.count"]].rename({"edge_media_preview_like.count":"Likes", "edge_media_to_comment.count": "CommentCount"}, axis=1)
df["ImageNumber"] = df.index + 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [141]:
df

,Likes,CommentCount,ImageNumber
0,483475,80,1
1,654176,50,2
2,686658,33,3
3,843936,45,4
4,692580,31,5


In [142]:
#get all the json file keys
from collections import Mapping

def extract_paths(base_path, dd): 
    new_paths = [] 
    for key, value in dd.items(): 
        new_path = base_path + ('.' if base_path else '') + key 
        if isinstance(value, Mapping): 
            new_paths.extend(extract_paths(new_path, value)) 
        else: 
            new_paths.append(new_path) 
    return new_paths

extract_paths('', data_json)

['GraphImages',
 'GraphProfileInfo.created_time',
 'GraphProfileInfo.info.biography',
 'GraphProfileInfo.info.followers_count',
 'GraphProfileInfo.info.following_count',
 'GraphProfileInfo.info.full_name',
 'GraphProfileInfo.info.id',
 'GraphProfileInfo.info.is_business_account',
 'GraphProfileInfo.info.is_joined_recently',
 'GraphProfileInfo.info.is_private',
 'GraphProfileInfo.info.posts_count',
 'GraphProfileInfo.info.profile_pic_url',
 'GraphProfileInfo.username']

In [143]:
#extracting comment data from the photo
def extract_data(df):

  test_f = pd.DataFrame([], columns = [])
  for i in range (0,len(df)):
    #liste des data des commentaires
     exp=pd.json_normalize(df[i])
     test_f=test_f.append(exp)
  return(test_f)


In [144]:
#exctarct all comments data
final= pd.DataFrame([], columns = [])
prov=pd.DataFrame([], columns = [])
complimentary_data=pd.DataFrame([], columns = [])
for j in range(0,len(data)):
  img_data=data_json["GraphImages"][j]
  comment_data=img_data["comments"]["data"]
  prov=extract_data(comment_data)
  prov["Img_Number"]=j+1
  final=final.append(prov)
  exp=pd.json_normalize(img_data)
  complimentary_data=complimentary_data.append(exp)
complimentary_data.rename(columns={'imgNumber': 'Img_Number'},inplace=True)
  


In [145]:
final

,created_at,id,text,owner.id,owner.profile_pic_url,owner.username,Img_Number
0,1619023963,18209883163069294,💪🏼💪🏼,20740995,https://instagram.ftun9-1.fna.fbcdn.net/v/t51....,sergiroberto,1
0,1619023981,18114517408211027,"🙏🏻 Deus não erra, não falha, Ele sabe de todas...",268668518,https://instagram.ftun9-1.fna.fbcdn.net/v/t51....,juliana_gilaberte,1
0,1619024010,18220882246023146,"Deus no comando, você vai voltar ainda mais fo...",242792330,https://instagram.ftun9-1.fna.fbcdn.net/v/t51....,isudaoliveira,1
0,1619024047,17868104936500119,Melhoras mago 🧙🏻‍♂️🙌,327818453,https://instagram.ftun9-1.fna.fbcdn.net/v/t51....,jp_sogomes,1
0,1619024226,17936515282506509,"Força e fé, pra cima 👊💜",206492939,https://instagram.ftun9-1.fna.fbcdn.net/v/t51....,indiracoutinho_,1
...,...,...,...,...,...,...,...
0,1613359639,17865476360320309,👏👏 Deus abençoe vocês sempre ❤️🙏,228479108,https://instagram.ftun9-1.fna.fbcdn.net/v/t51....,diegofariaoficial,5
0,1613371056,17892190312871812,👏🏾👏🏾👏🏾,257637026,https://instagram.ftun9-1.fna.fbcdn.net/v/t51....,t4_titi,5
0,1613426910,18141437446179500,Casalzão ❤️,2000235472,https://instagram.ftun9-1.fna.fbcdn.net/v/t51....,sigaoph,5
0,1613858973,17856095957504339,❤️❤️,3440329687,https://instagram.ftun9-1.fna.fbcdn.net/v/t51....,joao_paulo.__,5


In [152]:
#drop non needed columns:
final.drop(["created_at","owner.profile_pic_url"],axis=1,inplace=True)

In [153]:
final

,id,text,owner.id,owner.username,Img_Number
0,18209883163069294,💪🏼💪🏼,20740995,sergiroberto,1
0,18114517408211027,"🙏🏻 Deus não erra, não falha, Ele sabe de todas...",268668518,juliana_gilaberte,1
0,18220882246023146,"Deus no comando, você vai voltar ainda mais fo...",242792330,isudaoliveira,1
0,17868104936500119,Melhoras mago 🧙🏻‍♂️🙌,327818453,jp_sogomes,1
0,17936515282506509,"Força e fé, pra cima 👊💜",206492939,indiracoutinho_,1
...,...,...,...,...,...
0,17865476360320309,👏👏 Deus abençoe vocês sempre ❤️🙏,228479108,diegofariaoficial,5
0,17892190312871812,👏🏾👏🏾👏🏾,257637026,t4_titi,5
0,18141437446179500,Casalzão ❤️,2000235472,sigaoph,5
0,17856095957504339,❤️❤️,3440329687,joao_paulo.__,5


In [174]:
#renaming and exporting the data
final.rename(columns={'id': 'Comment ID',"text":"comment","owner.id":"commenter ID","owner.username":"commenter Name"},inplace=True)
final.reset_index(drop=True)
final.to_csv('/content/drive/MyDrive/Data_Science/Adnene task/instagram_data.csv',index=False,encoding="utf-8")

In [173]:
final

,Comment ID,comment,commenter ID,commenter Name,Img_Number
0,18209883163069294,💪🏼💪🏼,20740995,sergiroberto,1
0,18114517408211027,"🙏🏻 Deus não erra, não falha, Ele sabe de todas...",268668518,juliana_gilaberte,1
0,18220882246023146,"Deus no comando, você vai voltar ainda mais fo...",242792330,isudaoliveira,1
0,17868104936500119,Melhoras mago 🧙🏻‍♂️🙌,327818453,jp_sogomes,1
0,17936515282506509,"Força e fé, pra cima 👊💜",206492939,indiracoutinho_,1
...,...,...,...,...,...
0,17865476360320309,👏👏 Deus abençoe vocês sempre ❤️🙏,228479108,diegofariaoficial,5
0,17892190312871812,👏🏾👏🏾👏🏾,257637026,t4_titi,5
0,18141437446179500,Casalzão ❤️,2000235472,sigaoph,5
0,17856095957504339,❤️❤️,3440329687,joao_paulo.__,5


In [176]:
pip install pyarrow

In [178]:
import pyarrow as pa
import pyarrow.parquet as pq
final.to_parquet('/content/drive/MyDrive/Data_Science/Adnene task/instagram_data.parquet', engine='fastparquet')